Імпортувати основні бібліотеки

In [164]:
import pandas as pd
from pathlib import Path
import re




In [165]:
path = Path("data/Кладовище.xlsx")

dfs = []
with pd.ExcelFile(path, engine="openpyxl") as xls:
    for sh in xls.sheet_names:
        # читаємо як є, усі стовпці, все як текст
        df = pd.read_excel(
            xls,
            sheet_name=sh,
            header=None,
            dtype="string",
            keep_default_na=True
        )

        # якщо порожній аркуш — пропускаємо
        if df.empty:
            continue

        # залишаємо максимум перших 5 стовпців; якщо менше — доповнимо
        df = df.iloc[:, :5]                            # може бути 1..5 колонок
        need_cols = 5 - df.shape[1]
        if need_cols > 0:
            # додаємо відсутні колонки праворуч
            for _ in range(need_cols):
                df[df.shape[1]] = pd.NA                # додаємо порожній стовпець

        # тепер гарантовано 5 колонок → даємо імена
        df.columns = ["number", "name", "dates", "teritory", "row_list"]

        # якщо в Excel були merged-клітинки — тягнемо значення згори вниз
        df[["number", "name", "dates", "teritory", "row_list"]] = \
            df[["number", "name", "dates", "teritory", "row_list"]].ffill()

        # не перезаписуємо row_list; кладемо назву аркуша в окрему колонку
        df["sheet"] = sh

        # відкинути повністю порожні рядки
        if not df.dropna(how="all").empty:
            dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
df_all.drop(columns=["number"], inplace=True)

print(df_all.shape)
print(df_all.tail(10))

(9479, 5)
                      name      dates      teritory row_list   sheet
9469       Журавлевич Іван  1922-1983  біля туалету      NaN  Ряд 95
9470        Пронін Валерій  1954-2020  біля туалету      NaN  Ряд 95
9471        Пазіна Наталія  1918-1983  біля туалету      NaN  Ряд 95
9472  Загорський Олександр  1918-1983  біля туалету      NaN  Ряд 95
9473  Головенко Олександра  1908-1983  біля туалету      NaN  Ряд 95
9474      Починка Григорій  1915-1983  біля туалету      NaN  Ряд 95
9475              невідомо  1915-1983  біля туалету      NaN  Ряд 95
9476         Шудра Омельян  1913-1983  біля туалету      NaN  Ряд 95
9477          Васько Марія  1940-2009  біля туалету      NaN  Ряд 95
9478         Васько Сергій  1962-1983  біля туалету      NaN  Ряд 95


C:\Users\ПК\AppData\Local\Temp\ipykernel_2676\2858186220.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat(dfs, ignore_index=True)


### Заповнити пробіли псевдоданими

In [166]:
# Заповнити пропуски псевдоданими
# df_all['full_name'] = df_all["full_name"].fillna("John_Dou") #Заповнити пропуски в іменах
df_all['teritory'] = df_all['teritory'].fillna("To Be Confirmed") 
df_all.isna().sum()



name           1
dates         24
teritory       0
row_list    9479
sheet          0
dtype: int64

In [167]:
# Додати колонку з кількістю слів в датафрейм, в майбутньому для розділення імені

word_pattern = r"[A-Za-zА-Яа-яІіЇїЄєҐґ’']+" #будь яка послідовність кирилиці або латиниці є словом

# колонка з кількістю слів
df_all["word_count"] = (
    df_all["name"]  
      .fillna("") #Заповнити null щоб не падало (просто страховка, вище замінив на J Dou])
      .str.replace(r"\s+", " ", regex=True).str.strip() # прибрати зайві пробіли
      .str.findall(word_pattern) # знайти матчі
      .str.len() # порахувати матчі
)

df_all["need_correct"] = df_all["word_count"] <= 1

df_all.to_excel("pre_data.xlsx")


In [ ]:
# Проставити імя та призвіще
norm = (
    df_all["name"]
      .fillna("")
      .str.replace(r"\s+", " ", regex=True).str.strip()
)


df_all["second_name"]  = norm.str.extract(rf"^({word_pattern})", expand=False)
df_all["first_name"] = norm.str.extract(rf"^(?:{word_pattern})\s+({word_pattern})", expand=False)

with pd.option_context("display.max_rows", None):
    print(df_all["first_name"].value_counts())
    


first_name
Марія             481
Іван              352
Володимир         292
Микола            213
Михайло           174
Катерина          173
Ольга             167
Олександр         166
Василь            166
Петро             158
Ганна             156
Анна              144
Надія             136
Степан            131
Олена             118
Віра              102
Йосип              99
Софія              97
Андрій             93
Анастасія          91
Тетяна             87
Григорій           86
Федір              85
Антоніна           79
Сергій             79
Павло              78
Антон              77
Олексій            70
Олександра         69
Ніна               68
Євгенія            67
Анатолій           65
Галина             65
Євдокія            62
Віктор             60
Наталія            53
Валентина          52
Дмитро             52
Лідія              51
Варвара            47
Людмила            45
Миколай            38
Ксенія             38
Юлія               38
Любов              38